In [1]:
import os
import copy
import time
import pickle
import numpy as np
import yaml
from tqdm import tqdm

import torch
from torch.utils.data import DataLoader, Dataset
import torch.nn as nn
import torch.nn.functional as F

from generate_synthetic import generate_synthetic, train_val_dataloader

torch.manual_seed(0)
torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = False
np.random.seed(0)

In [2]:
config = dict()
config['local_batch_size'] = 10
config['local_iters'] = 1            # E
config['global_iters'] = 200         # T

config['num_devices'] = 30           # p
config['num_active_devices'] = 10

config['lr'] = 0.1
config['num_classes'] = 10
config['device'] = 'cuda:0'

config['iid'] = 1
config['alpha'] = 0
config['beta'] = 0
config['gamma'] = 0.1
config['dimension'] = 60

config_path = '../config/synthetic_iid.yaml' 
with open(config_path, 'w') as f:
    yaml.dump(config, f)

In [3]:
with open(config_path, 'r') as f:
    config = yaml.load(f, Loader=yaml.FullLoader)
    print(config)

{'alpha': 0, 'beta': 0, 'device': 'cuda:0', 'dimension': 60, 'gamma': 0.1, 'global_iters': 200, 'iid': 1, 'local_batch_size': 10, 'local_iters': 1, 'lr': 0.1, 'num_active_devices': 10, 'num_classes': 10, 'num_devices': 30}


In [4]:
# define training model
class MLP(nn.Module):
    def __init__(self, dim_in, dim_out):
        super(MLP, self).__init__()
        self.layer_hidden = nn.Linear(dim_in, dim_out)

    def forward(self, x):
        x = self.layer_hidden(x)
        return F.log_softmax(x, dim=1)
    
device = torch.device(config['device'])
    
global_model = MLP(dim_in=config['dimension'], dim_out=config['num_classes']).to(device)
local_model_list = [
    MLP(dim_in=config['dimension'], dim_out=config['num_classes']).to(device)
    for _ in range(config['num_devices'])
]

criterion = nn.NLLLoss().to(device)

In [5]:
# generate training data
X_split, y_split, weight_per_user = generate_synthetic(alpha=config['alpha'], beta=config['beta'], iid=config['iid'], 
                                      num_user=config['num_devices'], dimension=config['dimension'],
                                      num_class=config['num_classes'])

trainloader_list, validloader_list = [], []
trainloader_iterator_list, validloader_iterator_list = [], []
for local_id in range(config['num_devices']):
    trainloader, validloader = train_val_dataloader(X_split, y_split, local_id, batch_size=config['local_batch_size'])
    trainloader_list.append(trainloader)
    validloader_list.append(validloader)
    
    trainloader_iterator_list.append(iter(trainloader_list[local_id]))
    validloader_iterator_list.append(iter(validloader_list[local_id]))

0-th users has 1909 exampls
1-th users has 171 exampls
2-th users has 436 exampls
3-th users has 4876 exampls
4-th users has 2337 exampls
5-th users has 57 exampls
6-th users has 415 exampls
7-th users has 90 exampls
8-th users has 94 exampls
9-th users has 174 exampls
10-th users has 122 exampls
11-th users has 1050 exampls
12-th users has 300 exampls
13-th users has 119 exampls
14-th users has 182 exampls
15-th users has 156 exampls
16-th users has 1133 exampls
17-th users has 86 exampls
18-th users has 152 exampls
19-th users has 59 exampls
20-th users has 50 exampls
21-th users has 251 exampls
22-th users has 357 exampls
23-th users has 62 exampls
24-th users has 5162 exampls
25-th users has 52 exampls
26-th users has 109 exampls
27-th users has 87 exampls
28-th users has 1220 exampls
29-th users has 1081 exampls


In [6]:
def inference(model, dataloader):
    """ Returns the inference accuracy and loss.
    """

    model.eval()
    total, correct = 0.0, 0.0
    loss = list()
    
    for batch_idx, (images, labels) in enumerate(dataloader):
        images, labels = images.to(device), labels.to(device)

        # Inference
        outputs = model(images)
        batch_loss = criterion(outputs, labels)
        loss += [batch_loss.item()]

        # Prediction
        _, pred_labels = torch.max(outputs, 1)
        pred_labels = pred_labels.view(-1)
        correct += torch.sum(torch.eq(pred_labels, labels)).item()
        total += len(labels)

    accuracy = correct/total
    loss = sum(loss)/len(loss)
    return accuracy, loss

In [9]:
def average_state_dicts(w, weight):
    """
    Returns the average of the weights or gradients.
    """
    weight = weight/sum(weight)
    
    w_avg = copy.deepcopy(w[0])
    for key in w_avg.keys():
        w_avg_ = torch.zeros_like(w_avg[key])
        for i in range(len(w)):
            w_avg[key] = w_avg[key] + w[i][key]*weight[i]
    return w_avg

In [11]:
for global_iters in range(config['global_iters']):
    
    activate_devices = np.arange(config['num_devices'])
    
    # get the local grad for each device
    local_grad_list = []
    for local_id in activate_devices:
        # load single mini-batch
        try:
            inputs, labels = next(trainloader_iterator_list[local_id])
        except StopIteration:
            trainloader_iterator_list[local_id] = iter(trainloader_list[local_id])
            inputs, labels = next(trainloader_iterator_list[local_id])
    
        # train local model
        inputs, labels = inputs.to(device), labels.to(device)
        local_model_list[local_id].train()
        local_model_list[local_id].zero_grad()
        log_probs = local_model_list[local_id](inputs)
        loss = criterion(log_probs, labels)
        loss.backward()
        
        local_grads = dict()
        for key, val in local_model_list[local_id].named_parameters():
            local_grads[key] = val.grad.data
        
        local_grad_list.append(local_grads)
        
    # update local models
    if global_iters % config['local_iters'] == 0:
        avg_local_grad = average_state_dicts(local_grad_list, weight_per_user[activate_devices])
        
    # update local weight
    for local_id in activate_devices:
        local_weight = local_model_list[local_id].state_dict()
        local_grad = local_grad_list[local_id]
        for key in local_weight.keys():
            local_weight[key] = local_weight[key] - config['lr'] * (avg_local_grad[key] * config['gamma'] + local_grad[key] * (1-config['gamma']))
        local_model_list[local_id].load_state_dict(local_weight)

    # 
    local_weight_list = [local_model.state_dict() for local_model in local_model_list]
    avg_local_weight = average_state_dicts(local_weight_list, weight_per_user)
    global_model.load_state_dict(avg_local_weight)
    for local_id in range(config['num_devices']):
        local_model_list[local_id].load_state_dict(avg_local_weight)